In [1]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


# hyperparameters

In [3]:
#data
DATASET = 'cifar10'
class_num   = 10
test_size   = None
train_size  = 45000
image_shape = None

if DATASET =='mnist':
    image_shape = (28, 28, 1)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

#training
batch_size = 256
epochs = 200

In [4]:
x_train_all, y_train_all, x_test_all, y_test_all = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                                  do_flatten_and_normalize=False))

In [5]:
# shuffle
seed = 0
x_train_all, y_train_all = shaffle(x_train_all, y_train_all, seed)

In [6]:
# down sample
x_train = x_train_all[:train_size]
y_train = y_train_all[:train_size]

x_valid = x_train_all[train_size:]
y_valid = y_train_all[train_size:]

x_test = x_test_all[:test_size]
y_test = y_test_all[:test_size]

In [7]:
x_train, x_valid, x_test = x_train.reshape((-1, *image_shape)), x_valid.reshape((-1, *image_shape)), x_test.reshape((-1, *image_shape))

In [8]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(
    100000
).batch(
    batch_size
).prefetch(10)

In [9]:
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid)).batch(batch_size)

In [10]:
layers = tf.keras.layers

In [11]:
img_input = layers.Input(shape=image_shape)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(img_input)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
out = layers.Dense(10, kernel_initializer=tf.keras.initializers.GlorotNormal())(x)

model = tf.keras.Model(inputs=img_input, outputs=out)

In [12]:
def scheduler(epoch, lr):
    if epoch < 10:
        return 1e-3
    elif epoch < 90:
        return 1e-2
    elif epoch < 150:
        return 1e-3
    else:
        return 1e-4

In [13]:
model.compile(optimizer=tf.keras.optimizers.SGD(momentum=0.9),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [14]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [15]:
# model.fit(x=train_ds, validation_data=valid_ds, epochs=epochs, callbacks=[callback])

In [16]:
model.load_weights('./model_weights/vgg19_train_without_DA=all_ce.h5')

In [17]:
model.evaluate(x=x_test, y=y_test_all, verbose=0)

[1.8929608956336974, 0.7679]

In [19]:
file_list = ['cifar-fgsm-eps-0.03-time-500000.npy',
             'cifar-fgsm-eps-0.03-time-100000.npy',
             'cifar-fgsm-eps-0.03-time-1000000.npy',
             'cifar-fgsm-eps-0.03-time-2300000.npy',
             './npy/batch_NTK_simple.npy',
             './npy/cifar-untargeted-cifar-nn-grey-box-train=all-ce.npy',
             './npy/batch_NTK_cnn19.npy',
             './npy/cifar-untargeted-cifar-nn-grey-box-cnn19-train=all-ce.npy',
             "./npy/batch_NTK_cnn19_decrease_variance.npy",
             "./npy/batch_NTK_cnn19_increase_variance.npy",
             "./npy/batch_NTK_simple_decrease_variance.npy",
             "./npy/batch_NTK_simple_increase_variance.npy"]

for f in file_list:
    tmp = onp.load(f)
    print('evaluating file: %s'%(f))
    result = model.evaluate(tmp[:512], y_test[:512], verbose=0)
    print(result)

evaluating file: cifar-fgsm-eps-0.03-time-500000.npy
[2.6013945192098618, 0.703125]
evaluating file: cifar-fgsm-eps-0.03-time-100000.npy
[2.601825535297394, 0.703125]
evaluating file: cifar-fgsm-eps-0.03-time-1000000.npy
[2.6013945192098618, 0.703125]
evaluating file: cifar-fgsm-eps-0.03-time-2300000.npy
[2.6013945192098618, 0.703125]
evaluating file: ./npy/batch_NTK_simple.npy
[2.6013945192098618, 0.703125]
evaluating file: ./npy/cifar-untargeted-cifar-nn-grey-box-train=all-ce.npy
[3.2071173191070557, 0.6777344]
evaluating file: ./npy/batch_NTK_cnn19.npy
[2.448627844452858, 0.7050781]
evaluating file: ./npy/cifar-untargeted-cifar-nn-grey-box-cnn19-train=all-ce.npy
[2.6360759288072586, 0.7089844]
evaluating file: ./npy/batch_NTK_cnn19_decrease_variance.npy
[2.45619910210371, 0.70703125]
evaluating file: ./npy/batch_NTK_cnn19_increase_variance.npy
[2.440982148051262, 0.70703125]
evaluating file: ./npy/batch_NTK_simple_decrease_variance.npy
[2.63244292140007, 0.70703125]
evaluating file:

In [19]:
file_list = ['batch_NTK_simple_decrease_variance.npy',
             'batch_NTK_simple_increase_variance.npy',
             'batch_NTK_CNN10_decrease_variance.npy',
             'batch_NTK_CNN10_increase_variance.npy']

for f in file_list:
    tmp = onp.load(f)
    print('evaluating file: %s'%(f))
    result = model.evaluate(tmp, y_test[:2048], verbose=0)
    print(result)

evaluating file: batch_NTK_simple_decrease_variance.npy
[2.8453345438465476, 0.6621094]
evaluating file: batch_NTK_simple_increase_variance.npy
[2.83843188919127, 0.6586914]
evaluating file: batch_NTK_CNN10_decrease_variance.npy
[2.6379560194909573, 0.67626953]
evaluating file: batch_NTK_CNN10_increase_variance.npy
[2.6244671512395144, 0.67578125]


In [17]:
# model.save('./model_weights/simple_cnn_train=all_ce.h5')

In [ ]:
# with pooling, acc can reach 72.2%
# w.o. pooling, acc is at most 67%

In [18]:
tmp = onp.load('./npy/cifar-eps-time-any-npy/cifar-fgsm-eps-0.03-time-None.npy')
print("==========NTK============")
model.evaluate(tmp, y_test[:128], verbose=0)

==========NTK============


[1.721642017364502, 0.796875]

In [19]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=4096-ce.npy')
print("==========CE============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========CE============


[1.4565632166631985, 0.82714844]

In [20]:
tmp = onp.load('./npy/cifar-fgsm-eps-0.03-time-None-nngp.npy')
model.evaluate(tmp, y_test[:128], verbose=0)

[1.5229312479496002, 0.828125]

In [21]:
tmp = onp.load('./npy/cifar-fgsm-eps-0.03-time-None-nngp.npy')
model.evaluate(tmp, y_test[:128], verbose=0)

[1.5229312479496002, 0.828125]

In [19]:
tmp = onp.load('./batch_NTK_cnn_19.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

[2.5074403285980225, 0.71826171875]

In [20]:
tmp = onp.load('./batch_NTK_simple.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

[2.7732131481170654, 0.6923828125]

In [21]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-cnn19-train=all-ce.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

[2.5643069744110107, 0.7158203125]

In [22]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=all-ce.npy')
print("==========small============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========small============


[3.2448506355285645, 0.6474609375]